# Taller DNS
En este taller vamos a trabajar sobre el protocolo DNS. Utilizaremos la biblioteca [**Scapy**](https://scapy.readthedocs.io/en/latest/) de Python, que nos permite armar paquetes de forma super sencilla, usando las capas que probamos. Ejecuten la siguiente celda para instalarlo en este collab de ahora:



In [44]:
%pip install scapy
import scapy.all as scapy
from scapy.layers.inet import IP
from scapy.layers.inet import TCP
from scapy.layers.inet import UDP
from scapy.layers.dns import DNS, DNSQR

Note: you may need to restart the kernel to use updated packages.


A continuación, importemos Scapy y armemos nuestro primer paquete. Vamos a crear un paquete IP, que pertenece a la capa de Red (todavía no vimos en profundidad esta capa, pero nos vamos a adelantar un poco a la teoría para poder completar este taller).

In [45]:
pck = IP() # Creamos un paquete inicializado con valores default
pck.ttl = 64 # "Le definimos un TTl. Con esto ya podríamos simular un traceroute!
pck.dst = "www.google.com" # Le definimos una URL destino
pck

<IP  ttl=64 dst=Net("www.google.com/32") |>

# **Ejercicio 0**
Además de crear paquetes, también podemos enviarlos usando Scapy. En este caso, vamos a enviar un paquete con un TTL bajo para que el paquete no llegue a su destino, e inspeccionar la respuesta. Para este ejercicio entonces deberán:
- Crear un `paquete` con TTL bajo
- Enviarlo al destino usando la funcion `sr1(paquete)`
- Si obtienen una respuesta, mostrarla usando la funcion `show()`

In [49]:
paqueteTTL = 1
paqueteDST = "www.google.com"
paquete = IP(ttl=paqueteTTL, dst='8.8.8.8')

scapy.sr1(paquete)


Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


<IP  version=4 ihl=5 tos=0xc0 len=48 id=19170 flags= frag=0 ttl=64 proto=icmp chksum=0xadd3 src=192.168.0.1 dst=192.168.0.6 |<ICMP  type=time-exceeded code=ttl-zero-during-transit chksum=0xf4ff reserved=0 length=0 unused=0 |<IPerror  version=4 ihl=5 tos=0x0 len=20 id=1 flags= frag=0 ttl=1 proto=ip chksum=0xe92b src=192.168.0.6 dst=8.8.8.8 |>>>

Ahora ya sabemos como mandar paquetes IP (capa de Red).
Pero también podemos especificar qué protocolo de la capa superior quisieramos usar. A nivel transporte podemos elegir TCP o UDP.
Este tipo de indicaciones las podemos hacer usando la `"/"`.

In [42]:
import scapy.all as scapy
pck = IP(ttl = 64, dst = "www.google.com") / TCP() # Creamos un paquete en la capa de transporte (igualmente le podemos cambiar cosas de la capa de abajo! como el TTL)
pck.dst = "www.google.com" # Le definimos una URL destino
pck

<IP  frag=0 ttl=64 proto=tcp dst=Net("www.google.com/32") |<TCP  |>>

# **Ejercicio 1**
Deberán armar un paquete DNS y recibir la respuesta. En particular, resuelvan la direccion www.google.com  Para eso tendrán que:

    1- Crear un paquete IP. Su destino debe ser el de algún servidor DNS (Por ejemplo, el DNS publico de *google* es **8.8.8.8**)

    2- Asignarle un protocolo de capa de transporte a ese paquete. ¿Debería un request DNS ser UDP o TCP? ¿Que puerto de destino debería tener un request DNS?

    3- Al mismo paquete, deben modificale info a nivel aplicación (usen \ DNS() ).
    
    4- Inspeccionen las respuestas recibidas y analicen el paquete. ¿La IP en la respuesta correspondiente efectivamente a www.google.com? (Esto lo podemos verificar enviando un *ping* a la dirección, o ingresando a http://*ip_recibido* desde el navegador).

Al paquete DNS deberán definirle algunos campos. Revisen la [documentación](https://scapy.readthedocs.io/en/latest/api/scapy.layers.dns.html?highlight=dns#scapy.layers.dns.DNS.fields_desc) de los paquetes DNS para determinar cómo usar la función `DNSQR` para modificar campos de su query. En particular, se necesitará modificar el campo QD (cuerpo) del paquete, formado por:

    a) qname: el nombre a resolver.
    b) qtype: si se resuelve un mail o una página web.





In [55]:
paquete = IP(ttl=64, dst='8.8.8.8') / UDP(dport=53) / DNS(rd = 1, qd = DNSQR(qname='www.google.com', qtype='A'))
respuesta = scapy.sr1(paquete, verbose = 0)
respuesta.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 76
  id        = 3157
  flags     = 
  frag      = 0
  ttl       = 119
  proto     = udp
  chksum    = 0x668e
  src       = 8.8.8.8
  dst       = 192.168.0.6
  \options   \
###[ UDP ]### 
     sport     = domain
     dport     = domain
     len       = 56
     chksum    = 0x4ae4
###[ DNS ]### 
        id        = 0
        qr        = 1
        opcode    = QUERY
        aa        = 0
        tc        = 0
        rd        = 1
        ra        = 1
        z         = 0
        ad        = 0
        cd        = 0
        rcode     = ok
        qdcount   = 1
        ancount   = 1
        nscount   = 0
        arcount   = 0
        \qd        \
         |###[ DNS Question Record ]### 
         |  qname     = 'www.google.com.'
         |  qtype     = A
         |  qclass    = IN
        \an        \
         |###[ DNS Resource Record ]### 
         |  rrname    = 'www.google.com.'
         |  type      = A
     

## Ejercicio 2

Como vieron en la clase teórica, las consultas DNS pueden resolverse de manera iterativa. Es decir que el servidor (**8.8.8.8** en el caso del ejercicio anterior) va a realizar una consulta a cada *nameserver* que especifica una zona en la url.

Por ejemplo, en la url www.google.com primero se va a resolver el servidor correspondiente a **.com** y luego, a este servidor se le va a preguntar por la dirección correspondiente al nameserver autoritativo de **.google**.

Pero, ¿qué servidor sabe la dirección del nameserver **.com** originalmente?. Para ello existen los llamados **Root Name Servers**. Estos tienen direcciones IP constantes, con lo cual no debemos resolverla y por lo tanto son siempre el punto de partida para cualquier resolución. Pueden visitar este [link](https://en.wikipedia.org/wiki/Root_name_server) para ver la lista completa de **RNSs**.

    1- Enviar misma consulta del ejercicio anterior pero a un Root name server. ¿Nos da el mismo resultado que antes?
    2- Realizar las consultas iterativas para obtener la dirección IP de la url *www.google.com*.

In [25]:
# RESOLVER ACA